In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(f"Total Number of Rows: {len(full_data_rows_list)}")

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total Number of Rows: 8056


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [4]:
# Connection to a Cassandra on local machine 

from cassandra.cluster import Cluster
import cassandra
try:  
    # Create Cassandra Cluster
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    print('Cluster created')
except Exception as e:
    print(e)

Cluster created


In [5]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
    print("Keyspace created")
except Exception as e:
    print(e)

Keyspace created


In [6]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
    print("Keyspace set")
except Exception as e:
    print(e)

Keyspace set


## This file sets up the database to answer the following queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [7]:
# Exploring dataset
import pandas as pd
df = pd.read_csv("event_datafile_new.csv")
print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB
None


Four cells per query:
- Creation of table
- Insertion of Data
- Select statement against table showing data populated
- Query statement against table (original question)

### QUERY 1: Creation of Table

In [77]:
table_name_1 = "session_table"

drop_session_table_query = f"""DROP TABLE IF EXISTS {table_name_1}"""

create_session_table_query = f"""CREATE TABLE IF NOT EXISTS {table_name_1}
        (sessionId int, 
         itemInSession int, 
         artist text, 
         song text, 
         length float, 
         PRIMARY KEY (sessionId, itemInSession)
         )
        """

try:
    session.execute(drop_session_table_query)
    session.execute(create_session_table_query)
    print(f"{table_name_1} created successfully")
    
except Exception as e:
    print(f"[ERROR] Create Session Table Query: {e}")
                    

session_table created successfully


### QUERY 1: Insertion of Data

In [78]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {table_name_1} (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
print(f"{table_name_1} populated successfully")

session_table populated successfully


#### QUERY 1: Select statement to check table

In [79]:
query = f"SELECT * FROM {table_name_1} LIMIT 10"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)


Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')
Row(sessionid=23, iteminsession=1, artist='Octopus Project', length=250.95791625976562, song='All Of The Champs That Ever Lived')
Row(sessionid=23, iteminsession=2, artist='Tegan And Sara', length=180.06158447265625, song='So Jealous')
Row(sessionid=23, iteminsession=3, artist='Dragonette', length=153.39056396484375, song='Okay Dolores')
Row(sessionid=23, iteminsession=4, artist='Lil Wayne / Eminem', length=229.58975219726562, song='Drop The World')
Row(sessionid=23, iteminsession=5, artist="Soulja Boy Tell'em", length=201.1162872314453, song='Let Me Get Em')
Row(sessionid=23, iteminsession=6, artist='Bodo Wartke', length=645.2763061523438, song='Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃ\x83Â¶sisch_ Italienisch_ Spanisch_ HollÃ\x83Â¤ndisch_ Japanisch_ Russisch_ Griechisch_ Klingonisch_ Hessisch)')
Row(sessionid=23, iteminsession=7, artist='Evanescence', l

### Query 1: Answer to original Query / Question

In [80]:
# QUERY 1 ANSWER
query = f"SELECT artist, song, length from {table_name_1} WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### QUERY 2: Creation of Table

In [81]:
table_name_2 = "user_session"

drop_user_session_query = f"""DROP TABLE IF EXISTS {table_name_2}"""

create_user_session_query = f"""CREATE TABLE IF NOT EXISTS {table_name_2}
        (userId int, 
         sessionId int, 
         artist text, 
         song text, 
         firstName text, 
         lastName text,
         itemInSession int,
         PRIMARY KEY ((userId, sessionId), itemInSession)
         )
        """

try:
    session.execute(drop_user_session_query)
    session.execute(create_user_session_query)
    print(f"{table_name_2} created successfully")
    
except Exception as e:
    print(f"[ERROR] Create Session Table Query: {e}")
    

user_session created successfully


### QUERY 2: Insertion of Data

In [82]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {table_name_2} (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))
        
print(f"{table_name_2} populated successfully")

user_session populated successfully


### QUERY 2: Select statement to check table

In [83]:
query = f"SELECT * FROM {table_name_2} LIMIT 10"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', firstname='Emily', lastname='Benson', song='Sad Statue')
Row(userid=58, sessionid=768, iteminsession=1, artist='Ghostland Observatory', firstname='Emily', lastname='Benson', song='Stranger Lover')
Row(userid=58, sessionid=768, iteminsession=2, artist='Evergreen Terrace', firstname='Emily', lastname='Benson', song='Zero')
Row(userid=85, sessionid=776, iteminsession=2, artist='Deftones', firstname='Kinsley', lastname='Young', song='Head Up (LP Version)')
Row(userid=85, sessionid=776, iteminsession=3, artist='The Notorious B.I.G.', firstname='Kinsley', lastname='Young', song='Playa Hater (Amended Version)')
Row(userid=85, sessionid=776, iteminsession=4, artist='Orchestral Manoeuvres In The Dark', firstname='Kinsley', lastname='Young', song='Stay (The Black Rose And The Universal Wheel)')
Row(userid=85, sessionid=776, iteminsession=5, artist='Vangelis', firstname='Kinsley', lastname='Young', song="Roxane's Veil")
Row

### Query 2: Answer to original Query / Question

In [84]:
# QUERY 2 ANSWER
query = f"SELECT artist, song, firstname, lastname from {table_name_2} WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### QUERY 3: Creation of Table

In [85]:
table_name_3 = "user_songs"

drop_user_songs_query = f"""DROP TABLE IF EXISTS {table_name_3}"""

create_user_songs_query = f"""CREATE TABLE IF NOT EXISTS {table_name_3}
        (sessionId int, 
         song text, 
         firstName text, 
         lastName text,
         PRIMARY KEY (sessionId, song)
         )
        """

try:
    session.execute(drop_user_songs_query)
    session.execute(create_user_songs_query)
    print(f"{table_name_3} created successfully")
    
except Exception as e:
    print(f"[ERROR] Create Session Table Query: {e}")
    


user_songs created successfully


### QUERY 3: Insertion of Data

In [86]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {table_name_3} (sessionId, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[8]), line[9], line[1], line[4]))
        
print(f"{table_name_3} populated successfully")

user_songs populated successfully


### QUERY 3: Select statement to check table                    

In [87]:
query = f"SELECT * FROM {table_name_3} LIMIT 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(sessionid=23, song='3AM (LP Version)', firstname='Layla', lastname='Griffin')
Row(sessionid=23, song='All Of The Champs That Ever Lived', firstname='Layla', lastname='Griffin')
Row(sessionid=23, song='Boadicea', firstname='Layla', lastname='Griffin')
Row(sessionid=23, song='Bring Me To Life', firstname='Layla', lastname='Griffin')
Row(sessionid=23, song="Do That There (featuring Dude 'N Nem)", firstname='Layla', lastname='Griffin')
Row(sessionid=23, song='Dragula', firstname='Layla', lastname='Griffin')
Row(sessionid=23, song='Drop The World', firstname='Layla', lastname='Griffin')
Row(sessionid=23, song='Fairy Tale', firstname='Layla', lastname='Griffin')
Row(sessionid=23, song='Fearless', firstname='Layla', lastname='Griffin')
Row(sessionid=23, song='Girl', firstname='Layla', lastname='Griffin')


### Query 3: Answer to original Query / Question

In [88]:
# QUERY 3 ANSWER
query = f"SELECT firstname, lastname from {table_name_3} WHERE song = 'All Hands Against His Own' ALLOW FILTERING"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Sara Johnson
Tegan Levine
Jacqueline Lynch


In [ ]:
session.shutdown()
cluster.shutdown()